In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Determine the swap rate curve from discount curve
def swap_curve(P_zcb,delta=1):
    # Determine the swap rates from
    # compounding frequency = 1/delta 
    # time step is 1 by default
    from scipy.optimize import fsolve
    import numpy as np
    import pandas as pd
    P=list(P_zcb)
    swap_rates=[]
    for i,price in enumerate(P):
        temp_fun=(lambda t: np.sum([t*delta*discount for discount in P[:i+1]])+P[i]-1)
        c=fsolve(temp_fun,0.01)
        swap_rates.append(c[0])
    df=pd.DataFrame(swap_rates,index=np.arange(len(P)+1)[1:])
    df.index.name='Period'
    df.columns=['Swap_rate']
    return df

In [3]:
sr=swap_curve([0.9758,0.9527,0.9289,0.9050,0.8808,0.8565,0.8327,0.8090,0.7858,0.7628],delta=0.5)
sr

,Swap_rate
Period,
1,0.049600
2,0.049054
3,0.049766
4,0.050500
5,0.051344
6,0.052185
7,0.052839
8,0.053491
9,0.054042


In [4]:
# Determine the discount curve from swap curve
def discount_curve(swap_curve,delta=1):
    # Determine the discount curve from swap curve
    # compounding frequency = 1/delta 
    # time step is 1 by default
    # compounding frequncy is 1/delta 
    from scipy.optimize import fsolve
    import numpy as np
    import pandas as pd
    sr=list(swap_curve)
    discount_factor=[]
    for i,c in enumerate(sr):
        temp_fun=(lambda t: np.sum([discount*delta*c for discount in discount_factor])+t+t*delta*c-1)
        discount=fsolve(temp_fun,1)
        discount_factor.append(discount[0])
    df=pd.DataFrame(discount_factor,index=np.arange(len(sr)+1)[1:])
    df.index.name='Period'
    df.columns=['Discount Curve']
    return df

In [5]:
discount_curve(sr.values,delta=0.5)

,Discount Curve
Period,
1,0.9758
2,0.9527
3,0.9289
4,0.9050
5,0.8808
6,0.8565
7,0.8327
8,0.8090
9,0.7858


In [82]:
def myzcb(r,T,delta=1,compunding=0):
     # r is the percentage of interesrate 
     # r is in line with compounding frequency = 1/delta if compunding =1
     # if compounding=0 ,Continuously Compounding
     # if compounding=1, compounding 1/delta times a year.
     # delta is the time step between payments
    import numpy as np
    if compunding==0:
        return np.exp(-r*T)
    elif compunding==1:
        n=1/delta
        return np.power(1+r/n,-T*n)
    else:
        print("Please choose right compunding method")

In [83]:
myzcb(0.05,1,compunding=1)

0.9523809523809523

In [85]:
myzcb(0.055,2,compunding=0)

0.8958341352965282

In [86]:
# Linear system
A=np.array([[1,0],[0,1]])
b=np.array([1,3])
np.linalg.solve(A,b)

array([1., 3.])

In [87]:
myzcb(0.11,0.5)

0.9464851479534838

In [88]:
class Vasicek():
    def __init__(self,gamma,sigma,rstar):
        import numpy as np
        import pandas as pd

        self.gamma=gamma
        self.sigma=sigma
        self.rstar=rstar
    
    def B(self,tau):
        import numpy as np
        return 1/self.gamma*(1-np.exp(-self.gamma*tau))
    
    def A(self,tau):
        return (self.B(tau)-tau)*(self.rstar-self.sigma**2/(2*self.gamma**2))-self.sigma**2*self.B(tau)*self.B(tau)/(4*self.gamma)

    def Z(self,tau,r0):
        import numpy as np
        return np.exp(self.A(tau)-self.B(tau)*r0)
    
    def YTM(self,tau,r0):
        return -self.A(tau)/tau+self.B(tau)/tau*r0
        

In [89]:
vasicek1=Vasicek(0.4653,0.0221,0.0634)

In [90]:
vasicek1.B(10)

2.1286633137567903

In [91]:
vasicek1.A(10)

-0.49135336496544096

In [92]:
vasicek1.Z(10,0.055)

0.5442046830847762

In [93]:
vasicek1.YTM(10,0.055)

0.06084298472220645

In [94]:
class CIR():
    def __init__(self,gamma,astar,rstar):
        import numpy as np
        import pandas as pd

        self.gamma=gamma
        self.a=astar
        self.rstar=rstar
        self.phi=np.sqrt(self.gamma**2+2*self.a)
    
    def B(self,tau):
        import numpy as np
        B1=2*(np.exp(self.phi*tau)-1)
        B2=(self.gamma+self.phi)*(np.exp(self.phi*tau)-1)+2*self.phi
        return B1/B2
    
    def A(self,tau):
        A1=(self.gamma+self.phi)*(np.exp(self.phi*tau)-1)+2*self.phi
        A2=2*self.rstar*self.gamma/self.a*np.log(2*self.phi*np.exp((self.phi+self.gamma)*tau/2)/(A1))
        return A2
        
    def Z(self,tau,r0):
        import numpy as np
        return np.exp(self.A(tau)-self.B(tau)*r0)
    
    def YTM(self,tau,r0):
        return -self.A(tau)/tau+self.B(tau)/tau*r0
        

In [95]:
cir1=CIR(0.4653,1,0.0634)

In [96]:
cir1.B(10)

1.0234922961258646

In [97]:
cir1.A(10)

-0.2770805660968212

In [98]:
cir1.Z(10,0.055)

0.7165031449619667

In [99]:
cir1.YTM(10,0.055)

0.033337264238374374